# Explore here

In [5]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 

In [6]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv"
datos = pd.read_csv(url)
datos.head()

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0


In [4]:
datos.shape 

(891, 3)

In [7]:
datos = datos.drop(columns=["package_name"]) 
print(datos.shape)
datos.head()

(891, 2)


,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


In [8]:
datos["review"] = datos["review"].str.strip().str.lower()

In [9]:
# Separar variables predictoras (X) y variable objetivo (y)
X = datos["review"]  # Texto de las reseñas
y = datos["polarity"]  # Sentimiento (0: negativo, 1: positivo)


In [10]:
from sklearn.model_selection import train_test_split

# Dividir en 80% entrenamiento y 20% prueba (ajusta test_size si lo prefieres)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar tamaños
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

Train: (712,), Test: (179,)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorizar texto (convertir palabras a números)
vec_model = CountVectorizer(stop_words="english")
X_train_vec = vec_model.fit_transform(X_train).toarray()  # Entrenar y transformar train
X_test_vec = vec_model.transform(X_test).toarray()       # Solo transformar test

OPCION 1 PARA MODELOS MULTNOMIAL, BERNOULLI Y GAUSSIAN

In [29]:
#opcion 1 para los modelos multinomialNB, Bernoulli y Gaussian

from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import accuracy_score

# Modelos a comparar
models = {
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB(),
    "GaussianNB": GaussianNB()
}

# Entrenar y evaluar
for name, model in models.items():
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    print(f"{name} - Accuracy: {accuracy_score(y_test, y_pred):.4f}")

MultinomialNB - Accuracy: 0.8156
BernoulliNB - Accuracy: 0.7709
GaussianNB - Accuracy: 0.8045


OPCION 2 PARA EJECTUAR LOS MODELOS MULTINOMIAL, BERNOULLI Y GAUSSIAN

In [30]:
#opcion 2 para los modelos multinomialNB, Bernoulli y Gaussian

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report

# Vectorizar el texto
vectorizer = CountVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

# --- GaussianNB ---
gnb_model = GaussianNB()
gnb_model.fit(X_train_vec, y_train)
y_pred_test_gnb = gnb_model.predict(X_test_vec)
print(f"GaussianNB Accuracy: {accuracy_score(y_test, y_pred_test_gnb):.4f}")
print(classification_report(y_test, y_pred_test_gnb))

# --- MultinomialNB --- (MEJOR OPCIÓN PARA TEXTO)
mnb_model = MultinomialNB()
mnb_model.fit(X_train_vec, y_train)
y_pred_test_mnb = mnb_model.predict(X_test_vec)
print(f"MultinomialNB Accuracy: {accuracy_score(y_test, y_pred_test_mnb):.4f}")
print(classification_report(y_test, y_pred_test_mnb))

# --- BernoulliNB ---
bnb_model = BernoulliNB()
bnb_model.fit(X_train_vec, y_train)
y_pred_test_bnb = bnb_model.predict(X_test_vec)
print(f"BernoulliNB Accuracy: {accuracy_score(y_test, y_pred_test_bnb):.4f}")
print(classification_report(y_test, y_pred_test_bnb))

GaussianNB Accuracy: 0.8045
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       126
           1       0.69      0.62      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179

MultinomialNB Accuracy: 0.8156
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       126
           1       0.73      0.60      0.66        53

    accuracy                           0.82       179
   macro avg       0.79      0.75      0.77       179
weighted avg       0.81      0.82      0.81       179

BernoulliNB Accuracy: 0.7709
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       126
           1       0.70      0.40      0.51        53

    accuracy                           0.77       179
   macro avg       0.74      0.66      0

MODELO OPTIMIZADO 

In [31]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 200),
    "fit_prior": [True, False]
}

# We initialize the random search
random_search = RandomizedSearchCV(mnb_model, hyperparams, n_iter = 50, scoring = "accuracy", cv = 5, random_state = 42)
random_search

RandomizedSearchCV(cv=5, estimator=MultinomialNB(), n_iter=50,
                   param_distributions={'alpha': array([ 0.01      ,  0.06020101,  0.11040201,  0.16060302,  0.21080402,
        0.26100503,  0.31120603,  0.36140704,  0.41160804,  0.46180905,
        0.51201005,  0.56221106,  0.61241206,  0.66261307,  0.71281407,
        0.76301508,  0.81321608,  0.86341709,  0.91361809,  0.9638191 ,
        1.0140201 ,  1.06422111,  1.11442211,  1.1646231...
        8.54417085,  8.59437186,  8.64457286,  8.69477387,  8.74497487,
        8.79517588,  8.84537688,  8.89557789,  8.94577889,  8.9959799 ,
        9.0461809 ,  9.09638191,  9.14658291,  9.19678392,  9.24698492,
        9.29718593,  9.34738693,  9.39758794,  9.44778894,  9.49798995,
        9.54819095,  9.59839196,  9.64859296,  9.69879397,  9.74899497,
        9.79919598,  9.84939698,  9.89959799,  9.94979899, 10.        ]),
                                        'fit_prior': [True, False]},
                   random_state=42, scoring='accuracy')

In [32]:
random_search.fit(X_train_vec, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")

Best hyperparameters: {'fit_prior': False, 'alpha': np.float64(1.917638190954774)}


In [33]:
model = MultinomialNB(alpha=1.917638190954774, fit_prior=False)
model.fit(X_train_vec, y_train)

# 3. Predicción y evaluación
y_pred = model.predict(X_test_vec)  # <--- Usar X_test_vec
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8212290502793296


GUARDAR MODELOS EN CARPETA PROCESSED 

In [34]:
import joblib
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# 1. Asegurar que la carpeta 'processed' exista
os.makedirs("data/processed", exist_ok=True)

# 2. Entrenar el modelo (si no lo has hecho antes)
# Asumiendo que:
# - X_train_vec es tu texto ya vectorizado (con CountVectorizer/TfidfVectorizer)
# - y_train son tus etiquetas
best_model = MultinomialNB()
best_model.fit(X_train_vec, y_train)

# 3. Guardar el modelo
joblib.dump(best_model, "data/processed/naive_bayes_model.pkl")

# 4. Guardar el vectorizador (asegúrate de que vec_model está definido)
# Ejemplo si usas CountVectorizer:
vec_model = CountVectorizer(stop_words='english')
vec_model.fit(X_train)  # X_train es el texto sin vectorizar
joblib.dump(vec_model, "data/processed/text_vectorizer.pkl")

print("Modelo guardado en: data/processed/naive_bayes_model.pkl")
print("Vectorizador guardado en: data/processed/text_vectorizer.pkl")

Modelo guardado en: data/processed/naive_bayes_model.pkl
Vectorizador guardado en: data/processed/text_vectorizer.pkl


In [35]:
# Para cargarlos luego:
model_loaded = joblib.load("data/processed/naive_bayes_model.pkl")
vectorizer_loaded = joblib.load("data/processed/text_vectorizer.pkl")

# Ejemplo de uso:
new_text = ["This app is great"]
new_text_vec = vectorizer_loaded.transform(new_text)
prediction = model_loaded.predict(new_text_vec)

joblib.dump(best_model, "data/processed/naive_bayes_model_v1.pkl")

['data/processed/naive_bayes_model_v1.pkl']

COMPARATIVA CON OTROS MODELOS 

BAG OF WORDS 

In [36]:
# bag of words 

from sklearn.feature_extraction.text import CountVectorizer

# Crear vectorizador (considerando las 5000 palabras más frecuentes)
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

# Aplicar solo a datos de entrenamiento (evitar data leakage)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)  # Usar el mismo vocabulario

print("\nForma de la matriz Bag of Words (Train):", X_train_bow.shape)
print("Ejemplo de vector BoW:", X_train_bow[0].toarray())


Forma de la matriz Bag of Words (Train): (712, 3310)
Ejemplo de vector BoW: [[0 0 0 ... 0 0 0]]


ANALISIS DE PALABRAS CLAVES 

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
import re

# 1. Mejorar la función de limpieza de texto
def clean_text(text):
    # Eliminar caracteres no ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Eliminar símbolos y caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar espacios múltiples
    text = ' '.join(text.split())
    return text

# Aplicar limpieza a tus datos
datos['clean_review'] = datos['review'].apply(clean_text)

# 2. Configurar mejor el vectorizador
vectorizer = CountVectorizer(
    stop_words='english',
    token_pattern=r'\b[a-zA-Z]{3,}\b',  # Solo palabras de 3+ letras
    max_features=5000  # Limitar a las 5000 palabras más frecuentes
)

# 3. Volver a crear la matriz de términos
X_train_bow = vectorizer.fit_transform(X_train.apply(clean_text))
X_test_bow = vectorizer.transform(X_test.apply(clean_text))

# 4. Reentrenar el modelo
model = MultinomialNB()
model.fit(X_train_bow, y_train)

# 5. Nuevo análisis de palabras importantes
log_probs = model.feature_log_prob_
feature_names = vectorizer.get_feature_names_out()

word_importance = pd.DataFrame({
    'word': feature_names,
    'weight_negative': log_probs[0],
    'weight_positive': log_probs[1]
})

print("\nTop 10 palabras para reseñas POSITIVAS:")
print(word_importance[word_importance['word'].str.isalpha()]  # Filtrar solo palabras alfabéticas
      .sort_values('weight_positive', ascending=False)
      .head(10))

print("\nTop 10 palabras para reseñas NEGATIVAS:")
print(word_importance[word_importance['word'].str.isalpha()]  # Filtrar solo palabras alfabéticas
      .sort_values('weight_negative', ascending=False)
      .head(10))


Top 10 palabras para reseñas POSITIVAS:
        word  weight_negative  weight_positive
135      app        -4.083951        -4.089590
1156    game        -5.416178        -4.526307
1221   great        -5.932869        -4.694444
1202    good        -5.433878        -4.879587
1623    like        -5.196550        -5.045101
1697    love        -6.894280        -5.128483
273     best        -7.262005        -5.319538
3113     use        -5.225123        -5.346206
2307  really        -5.745658        -5.430764
1521    just        -5.196550        -5.460617

Top 10 palabras para reseñas NEGATIVAS:
        word  weight_negative  weight_positive
135      app        -4.083951        -4.089590
3088  update        -4.622948        -5.779070
1069     fix        -4.777098        -6.654539
2958    time        -5.040389        -5.821630
1905     new        -5.040389        -5.661287
832     dont        -5.064780        -5.624920
826   doesnt        -5.155165        -7.570830
1623    like        -5.19

VECTORIZACION TF-IDF 


In [38]:
#Vectorización TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

# Crear vectorizador TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Aplicar a los datos
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Entrenar modelo con TF-IDF
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

# Evaluar precisión
y_test_pred_tfidf = model_tfidf.predict(X_test_tfidf)
print("\nPrecisión con TF-IDF:", accuracy_score(y_test, y_test_pred_tfidf))


Precisión con TF-IDF: 0.7988826815642458


RANDOM FOREST 

In [39]:

# modelo de random forest

from sklearn.ensemble import RandomForestClassifier

# Entrenar Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vec, y_train)
rf_accuracy = accuracy_score(y_test, rf_model.predict(X_test_vec))

print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

Random Forest Accuracy: 0.7989


REGRESION LOGISTICA 

In [40]:
# modelo de regresión logistica

from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_vec, y_train)
lr_accuracy = accuracy_score(y_test, lr_model.predict(X_test_vec))

print(f"Regresión Logística Accuracy: {lr_accuracy:.4f}")

Regresión Logística Accuracy: 0.8324


COMPARATIVA ENTRE MODELOS 

In [42]:
#comparativa entre modelos naive bayes, random forest y regresion logistica

print(f"Naive Bayes: {accuracy_score(y_test, best_model.predict(X_test_vec)):.4f}")
print(f"Random Forest: {rf_accuracy:.4f}")
print(f"Regresión Logística: {lr_accuracy:.4f}")
print("\nPrecisión con TF-IDF:", accuracy_score(y_test, y_test_pred_tfidf))



Naive Bayes: 0.8156
Random Forest: 0.7989
Regresión Logística: 0.8324

Precisión con TF-IDF: 0.7988826815642458
